In [2]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import numpy as np

/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

pipe = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-fused")

/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
#empty array 
audio_array = np.empty([])

word_detected = False

#CLAP model's prediction function
def classify(audio):
    global audio_array, word_detected
    
    samplerate, array = audio
    
    live_array = np.array(array)
    audio_array = np.append(audio_array, live_array)
    
    dimension = audio_array.shape
    
    
    #removes 5 second of audio from array if it's longer than 5 seconds
    if((dimension[0]) > (240000)):
        audio_array = audio_array[-240000:]
    
    print(audio_array.shape)
    
    result = pipe(audio_array, candidate_labels=["silence", "left","right","reload"])
    df = pd.DataFrame(result)
    max = df.idxmax(numeric_only=True)
    index = int(max[0])
    word = df.iat[index,2]
    print(max)
    print(index)
    print(type(index))
    print(word)
    print(df)
    
    

    #if word is detected or if the audio reaches 5 seconds, 
    #then reset the audio array to empty
    if (word_detected == True):
        audio_array = np.empty([1,1])
        word_detected = False
    
    return df

In [5]:
#Settings for Gradio Interface
demo = gr.Interface(
    fn=classify,
    inputs=gr.Audio(sources=["microphone"],streaming=True),
    outputs=[gr.Dataframe()],
    live=True,
    
)

In [6]:
#Launches Gradio Interface
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(24001,)


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
/var/folders/16/7hh1zfjs7cq_gs48v5wmkg1r0000gn/T/ipykernel_42181/4227735203.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  index = int(max[0])
Traceback (most recent call last):
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py"

(120001,)


Traceback (most recent call last):
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1689, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1255, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/anyio/to_

...

Traceback (most recent call last):
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1689, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1255, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/anyio/to_

(240000,)


Traceback (most recent call last):
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1689, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1255, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/anyio/to_

(240000,)


Traceback (most recent call last):
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1689, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1255, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/anyio/to_